# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [102]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-43fd22e0aa-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Euro explorative search") 

Consider the following exploratory information need:

> investigate the Euro currency, which are the countries that currently use the Euro and which were their currencies before the Euro. Take a look also to other currencies and find which are the most used.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q38`| Italy   | node |





Also consider

```
wd:Q38 ?p ?obj .
```

is the BGP to retrieve all **properties of Italy**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for currency

2. Return the countries which currently use Euro

3. Return the countries belonging to the European Union, with a currency different from the Euro (return also their currency).

4. For each country that now uses Euro return which was the currencies used before

5. Return all types of dollar in the world (US Dollar, Australian Dollar, ecc...)

6. Return the currencies used in at least 2 different countries (return also the number of countries using each currency).

7. Consider the currencies obtained in task 6, for each currency return the number of people using that currency (hint: consider the maximum popolation available for each country).

## Task 1
Identify the BGP for currency

In [103]:
#STRATEGY: 
#For all the unknown nodes that I want to explore for the first time I will follow the following strategy: 
# - (1) explore all the outcoming object properties from the node and I will return the property IRI, name and the related object IRI and name 
# - (2) explore all the incoming object properties from the node and I will return the property IRI, name and the related object IRI and name 
# - (3) explore all the node data properties

#When I do one of the previous explorative queries, I will write: "Explore **node** by performing (1)" (1) is for the query that explore all the outcoming object properties ... and so on

#I will not do all the previous steps every time, if I find what I need, I will "stop" the exploration strategy

#NOTICE THAT:

#If I have to retrieve the BGP of a node I will do a simple query that selects only the specified node with the 
#specified URI and I retrieve also the label. Obviously the URI that I used is retrieved in the queries that were done
#before that one.


In [104]:
#explore the initial node by performing (1)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q38 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('pName', 'member of'), ('obj', 'http://www.wikidata.org/entity/Q488981'), ('oName', 'European Bank for Reconstruction and Development')]
[('p', 'http://www.wikidata.org/prop/direct/P2936'), ('pName', 'language used'), ('obj', 'http://www.wikidata.org/entity/Q33973'), ('oName', 'Sicilian')]
[('p', 'http://www.wikidata.org/prop/direct/P2184'), ('pName', 'history of topic'), ('obj', 'http://www.wikidata.org/entity/Q7791'), ('oName', 'history of Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P163'), ('pName', 'flag'), ('obj', 'http://www.wikidata.org/entity/Q42876'), ('oName', 'flag of Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P150'), ('pName', 'contains administrative territorial entity'), ('obj', 'http://www.wikidata.org/entity/Q1273'), ('oName', 'Tuscany')]
[('p', 'http://www.wikidata.org/prop/direct/P2936'), ('pName', 'language used'), ('obj', 'http://www.wikidata.org/entity/Q33845'), ('oName', 'Neapolitan')]

In [105]:
#retrieve Italy node instances

queryString = """
SELECT DISTINCT ?instance ?instanceName
WHERE {
   wd:Q38 wdt:P31 ?instance . 
   ?instance sc:name ?instanceName .
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('instance', 'http://www.wikidata.org/entity/Q3624078'), ('instanceName', 'sovereign state')]
[('instance', 'http://www.wikidata.org/entity/Q6256'), ('instanceName', 'country')]
2


In [106]:
#explore the initial node by performing (2)

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q38 .
   ?s sc:name ?sName .
   ?p sc:name ?pName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q26299851'), ('sName', 'Morion'), ('p', 'http://www.wikidata.org/prop/direct/P2647'), ('pName', 'source of material')]
[('s', 'http://www.wikidata.org/entity/Q26299835'), ('sName', 'Morion'), ('p', 'http://www.wikidata.org/prop/direct/P2647'), ('pName', 'source of material')]
[('s', 'http://www.wikidata.org/entity/Q7685456'), ('sName', 'Tarantella Napoletana'), ('p', 'http://www.wikidata.org/prop/direct/P1268'), ('pName', 'represents')]
[('s', 'http://www.wikidata.org/entity/Q60836687'), ('sName', 'associazione sportiva dilettantistica'), ('p', 'http://www.wikidata.org/prop/direct/P1001'), ('pName', 'applies to jurisdiction')]
[('s', 'http://www.wikidata.org/entity/Q3682203'), ('sName', 'Italian law codes'), ('p', 'http://www.wikidata.org/prop/direct/P1001'), ('pName', 'applies to jurisdiction')]
[('s', 'http://www.wikidata.org/entity/Q3586606'), ('sName', '2013 Italian presidential election'), ('p', 'http://www.wikidata.org/prop/direct/P1

In [107]:
#search for the Italian Currency from the results of explorative query (1)

queryString = """
SELECT DISTINCT ?p ?propName ?obj ?objName
WHERE {
   wd:Q38 ?p ?obj .
   ?p sc:name ?propName.
   ?obj sc:name ?objName . 
   FILTER (REGEX(?propName, "currency"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P38'), ('propName', 'currency'), ('obj', 'http://www.wikidata.org/entity/Q4916'), ('objName', 'euro')]
1


In [108]:
#explore the euro currency node by performing (1)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q4916 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pName', 'replaces'), ('obj', 'http://www.wikidata.org/entity/Q16068'), ('oName', 'Deutsche Mark')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pName', 'replaces'), ('obj', 'http://www.wikidata.org/entity/Q788472'), ('oName', 'Dutch guilder')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pName', 'replaces'), ('obj', 'http://www.wikidata.org/entity/Q203354'), ('oName', 'Finnish markka')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pName', 'replaces'), ('obj', 'http://www.wikidata.org/entity/Q270642'), ('oName', 'Irish pound')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q28795800'), ('oName', 'decimal currency')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q7778928'), ('oName', 'Category:Euro')]
[('p', 'http://www.wikidata.org/prop/direct/P

Final query for this task

In [109]:
#return from the previous query the currency BGP (I found that the Euro node is an instance of currency)

queryString = """
SELECT DISTINCT ?uri ?name
WHERE {
   ?uri ?p ?obj .
   # get the label
   ?uri sc:name ?name .
   FILTER(?uri = wd:Q8142)
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('uri', 'http://www.wikidata.org/entity/Q8142'), ('name', 'currency')]
1


## Task 2
Return the countries which currently use Euro

In [110]:
#explore the Euro Currency node (retrieved in the previous queries) by performing (1)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q4916 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pName', 'replaces'), ('obj', 'http://www.wikidata.org/entity/Q16068'), ('oName', 'Deutsche Mark')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pName', 'replaces'), ('obj', 'http://www.wikidata.org/entity/Q788472'), ('oName', 'Dutch guilder')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pName', 'replaces'), ('obj', 'http://www.wikidata.org/entity/Q203354'), ('oName', 'Finnish markka')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pName', 'replaces'), ('obj', 'http://www.wikidata.org/entity/Q270642'), ('oName', 'Irish pound')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q28795800'), ('oName', 'decimal currency')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q7778928'), ('oName', 'Category:Euro')]
[('p', 'http://www.wikidata.org/prop/direct/P

In [111]:
#explore the Euro Currency node by performing (2)

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q38 .
   ?s sc:name ?sName .
   ?p sc:name ?pName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q26299851'), ('sName', 'Morion'), ('p', 'http://www.wikidata.org/prop/direct/P2647'), ('pName', 'source of material')]
[('s', 'http://www.wikidata.org/entity/Q26299835'), ('sName', 'Morion'), ('p', 'http://www.wikidata.org/prop/direct/P2647'), ('pName', 'source of material')]
[('s', 'http://www.wikidata.org/entity/Q7685456'), ('sName', 'Tarantella Napoletana'), ('p', 'http://www.wikidata.org/prop/direct/P1268'), ('pName', 'represents')]
[('s', 'http://www.wikidata.org/entity/Q60836687'), ('sName', 'associazione sportiva dilettantistica'), ('p', 'http://www.wikidata.org/prop/direct/P1001'), ('pName', 'applies to jurisdiction')]
[('s', 'http://www.wikidata.org/entity/Q3682203'), ('sName', 'Italian law codes'), ('p', 'http://www.wikidata.org/prop/direct/P1001'), ('pName', 'applies to jurisdiction')]
[('s', 'http://www.wikidata.org/entity/Q3586606'), ('sName', '2013 Italian presidential election'), ('p', 'http://www.wikidata.org/prop/direct/P1

Final query for this task

In [112]:
#return all the countries that use Euro by using the country property (P17) of the Euro node retrieved in the previous queries, 
#note that I will also retrive all the micro states and principalities that use Euro

queryString = """
SELECT DISTINCT ?country ?nameCountry 
WHERE {
   wd:Q4916 wdt:P17 ?country . 
   ?country sc:name ?nameCountry
}
ORDER BY ?nameCountry
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('country', 'http://www.wikidata.org/entity/Q228'), ('nameCountry', 'Andorra')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('nameCountry', 'Austria')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('nameCountry', 'Belgium')]
[('country', 'http://www.wikidata.org/entity/Q229'), ('nameCountry', 'Cyprus')]
[('country', 'http://www.wikidata.org/entity/Q191'), ('nameCountry', 'Estonia')]
[('country', 'http://www.wikidata.org/entity/Q33'), ('nameCountry', 'Finland')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('nameCountry', 'France')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('nameCountry', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q41'), ('nameCountry', 'Greece')]
[('country', 'http://www.wikidata.org/entity/Q27'), ('nameCountry', 'Ireland')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('nameCountry', 'Italy')]
[('country', 'http://www.wikidata.org/entity/Q211'), ('nameCountry', 'Latvia')]
[('country', 'http://www.wikidat

## Task 3
Return the countries belonging to the European Union, with a currency different from the Euro (return also their currency).

In [113]:
#explore again the Italy node and retrieve all related objects related to Europe

queryString = """
SELECT DISTINCT ?p ?propName ?obj ?objName
WHERE {
   # bind something
   wd:Q38 ?p ?obj .
   # get the label
   ?p sc:name ?propName.
   ?obj sc:name ?objName
   FILTER(REGEX(?objName, "^[E,e]urop"))
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P2853'), ('propName', 'electrical plug type'), ('obj', 'http://www.wikidata.org/entity/Q1378312'), ('objName', 'Europlug')]
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('propName', 'continent'), ('obj', 'http://www.wikidata.org/entity/Q46'), ('objName', 'Europe')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('propName', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q46'), ('objName', 'Europe')]
[('p', 'http://www.wikidata.org/prop/direct/P421'), ('propName', 'located in time zone'), ('obj', 'http://www.wikidata.org/entity/Q5412122'), ('objName', 'Europe/Rome')]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('propName', 'member of'), ('obj', 'http://www.wikidata.org/entity/Q161549'), ('objName', 'European Coal and Steel Community')]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('propName', 'member of'), ('obj', 'http://www.wikidata.org/entity/Q1377612'), ('objName', 'European Payments Union')]
[('p', '

In [114]:
#explore the European Union node (Q458, retrieved in the previous query) by performing (1)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q458 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received'), ('obj', 'http://www.wikidata.org/entity/Q3323440'), ('oName', 'Princess of Asturias Award for Concord')]
[('p', 'http://www.wikidata.org/prop/direct/P6104'), ('pName', 'maintained by WikiProject'), ('obj', 'http://www.wikidata.org/entity/Q10858397'), ('oName', 'WikiProject European Union')]
[('p', 'http://www.wikidata.org/prop/direct/P457'), ('pName', 'foundational text'), ('obj', 'http://www.wikidata.org/entity/Q11146'), ('oName', 'Maastricht Treaty')]
[('p', 'http://www.wikidata.org/prop/direct/P530'), ('pName', 'diplomatic relation'), ('obj', 'http://www.wikidata.org/entity/Q223'), ('oName', 'Greenland')]
[('p', 'http://www.wikidata.org/prop/direct/P530'), ('pName', 'diplomatic relation'), ('obj', 'http://www.wikidata.org/entity/Q23681'), ('oName', 'Turkish Republic of Northern Cyprus')]
[('p', 'http://www.wikidata.org/prop/direct/P194'), ('pName', 'legislative body'), ('obj', 'http://www.wikida

In [115]:
#explore the European Union node by performing (2)

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q458 .
   ?s sc:name ?sName .
   ?p sc:name ?pName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q3246979'), ('sName', 'Template:Structural evolution of the European Union'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic')]
[('s', 'http://www.wikidata.org/entity/Q10739195'), ('sName', 'Template:European Union topics'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic')]
[('s', 'http://www.wikidata.org/entity/Q602875'), ('sName', 'federalisation of the European Union'), ('p', 'http://www.wikidata.org/prop/direct/P3005'), ('pName', 'valid in place')]
[('s', 'http://www.wikidata.org/entity/Q1485366'), ('sName', 'Directorate-General'), ('p', 'http://www.wikidata.org/prop/direct/P1001'), ('pName', 'applies to jurisdiction')]
[('s', 'http://www.wikidata.org/entity/Q31885465'), ('sName', 'Directive 2011/95/EU of the European Parliament and of the Council of 13 December 2011'), ('p', 'http://www.wikidata.org/prop/direct/P1001'), ('pName', 'applies to jurisdiction')]
[('s', '

In [116]:
#retrieve all the member countries of the European Union using the 'has part' property (P527) retrieved in the exploration phase (1) 
#of the Europian Union node

queryString = """
SELECT DISTINCT ?p ?propName ?obj ?objName
WHERE {
   wd:Q458 wdt:P527 ?obj .
   ?obj sc:name ?objName.
}
ORDER BY ?objName
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q40'), ('objName', 'Austria')]
[('obj', 'http://www.wikidata.org/entity/Q31'), ('objName', 'Belgium')]
[('obj', 'http://www.wikidata.org/entity/Q219'), ('objName', 'Bulgaria')]
[('obj', 'http://www.wikidata.org/entity/Q224'), ('objName', 'Croatia')]
[('obj', 'http://www.wikidata.org/entity/Q229'), ('objName', 'Cyprus')]
[('obj', 'http://www.wikidata.org/entity/Q213'), ('objName', 'Czech Republic')]
[('obj', 'http://www.wikidata.org/entity/Q35'), ('objName', 'Denmark')]
[('obj', 'http://www.wikidata.org/entity/Q191'), ('objName', 'Estonia')]
[('obj', 'http://www.wikidata.org/entity/Q33'), ('objName', 'Finland')]
[('obj', 'http://www.wikidata.org/entity/Q142'), ('objName', 'France')]
[('obj', 'http://www.wikidata.org/entity/Q183'), ('objName', 'Germany')]
[('obj', 'http://www.wikidata.org/entity/Q41'), ('objName', 'Greece')]
[('obj', 'http://www.wikidata.org/entity/Q28'), ('objName', 'Hungary')]
[('obj', 'http://www.wikidata.org/entity/Q22

Final query for this task

In [117]:
#retrieve all the countries in the UE that use a currency different from the Euro with their currency

queryString = """
SELECT DISTINCT ?country ?countryName ?currency ?curName
WHERE {
   wd:Q458 wdt:P527 ?country  .
   ?country sc:name ?countryName.
    
   ?country wdt:P38 ?currency . 
   ?currency sc:name ?curName .
   FILTER(?currency != wd:Q4916)
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q213'), ('countryName', 'Czech Republic'), ('currency', 'http://www.wikidata.org/entity/Q131016'), ('curName', 'Czech koruna')]
[('country', 'http://www.wikidata.org/entity/Q218'), ('countryName', 'Romania'), ('currency', 'http://www.wikidata.org/entity/Q131645'), ('curName', 'Romanian leu')]
[('country', 'http://www.wikidata.org/entity/Q219'), ('countryName', 'Bulgaria'), ('currency', 'http://www.wikidata.org/entity/Q172540'), ('curName', 'Bulgarian lev')]
[('country', 'http://www.wikidata.org/entity/Q224'), ('countryName', 'Croatia'), ('currency', 'http://www.wikidata.org/entity/Q26360'), ('curName', 'Croatian kuna')]
[('country', 'http://www.wikidata.org/entity/Q28'), ('countryName', 'Hungary'), ('currency', 'http://www.wikidata.org/entity/Q47190'), ('curName', 'forint')]
[('country', 'http://www.wikidata.org/entity/Q34'), ('countryName', 'Sweden'), ('currency', 'http://www.wikidata.org/entity/Q122922'), ('curName', 'Swedish krona

## Task 4
For each country that now uses Euro return which was the currencies used before

In [118]:
#use the replaces property P1365 of the Euro node retrieved during the exploration phase (1) of that node

queryString = """
SELECT DISTINCT ?currency ?currencyName
WHERE {
   wd:Q4916 wdt:P1365 ?currency  .
   ?currency sc:name ?currencyName .
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)


Results
[('currency', 'http://www.wikidata.org/entity/Q16068'), ('currencyName', 'Deutsche Mark')]
[('currency', 'http://www.wikidata.org/entity/Q788472'), ('currencyName', 'Dutch guilder')]
[('currency', 'http://www.wikidata.org/entity/Q203354'), ('currencyName', 'Finnish markka')]
[('currency', 'http://www.wikidata.org/entity/Q270642'), ('currencyName', 'Irish pound')]
[('currency', 'http://www.wikidata.org/entity/Q184172'), ('currencyName', 'French franc')]
[('currency', 'http://www.wikidata.org/entity/Q189097'), ('currencyName', 'peseta')]
[('currency', 'http://www.wikidata.org/entity/Q204992'), ('currencyName', 'Italian lira')]
[('currency', 'http://www.wikidata.org/entity/Q468474'), ('currencyName', 'Portuguese escudo')]
[('currency', 'http://www.wikidata.org/entity/Q1546443'), ('currencyName', 'Greek drachma')]
[('currency', 'http://www.wikidata.org/entity/Q133011'), ('currencyName', 'Latvian lats')]
[('currency', 'http://www.wikidata.org/entity/Q174467'), ('currencyName', 'Malt

In [119]:
#from the previous results I note that I'm also considering some micro-states or principalities in the results, that are in the Europian Union but are part of other states
#and I'm considering some state more time:I want to filter the results more (previously we have obtained 19 rows as results and now 23 for countries that use Euro)

queryString = """
SELECT DISTINCT ?countryName ?replaceCurrency ?currencyName
WHERE {
   wd:Q4916 wdt:P1365 ?replaceCurrency  .
   ?replaceCurrency sc:name ?currencyName .
   ?replaceCurrency wdt:P17 ?country . 
   ?country sc:name ?countryName 
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)


Results
[('countryName', 'Captaincy General of Puerto Rico'), ('replaceCurrency', 'http://www.wikidata.org/entity/Q189097'), ('currencyName', 'peseta')]
[('countryName', 'France'), ('replaceCurrency', 'http://www.wikidata.org/entity/Q184172'), ('currencyName', 'French franc')]
[('countryName', 'Kingdom of Italy'), ('replaceCurrency', 'http://www.wikidata.org/entity/Q204992'), ('currencyName', 'Italian lira')]
[('countryName', 'Germany'), ('replaceCurrency', 'http://www.wikidata.org/entity/Q16068'), ('currencyName', 'Deutsche Mark')]
[('countryName', 'Estonia'), ('replaceCurrency', 'http://www.wikidata.org/entity/Q129983'), ('currencyName', 'Estonian kroon')]
[('countryName', 'Latvia'), ('replaceCurrency', 'http://www.wikidata.org/entity/Q133011'), ('currencyName', 'Latvian lats')]
[('countryName', 'Slovakia'), ('replaceCurrency', 'http://www.wikidata.org/entity/Q178874'), ('currencyName', 'Slovak koruna')]
[('countryName', 'Slovenia'), ('replaceCurrency', 'http://www.wikidata.org/entit

In [120]:
#trying this as final query

queryString = """
SELECT DISTINCT ?countryName ?replacedCurrency ?currencyName
WHERE {
    #euro replace ?replaceCurrency
   wd:Q4916 wdt:P1365 ?replacedCurrency  .
   ?replacedCurrency sc:name ?currencyName .
   
   #?replacedCurrency was used in a country
   ?replacedCurrency wdt:P17 ?country . 
   
   #check that the country is in the European Union
   FILTER (?country = ?c){
   
        #return all the countries in the European Union
        SELECT DISTINCT ?c
        WHERE {
           wd:Q458 wdt:P527 ?c .
           
        }
   }
   
   ?country sc:name ?countryName . 
}
ORDER BY ?countryName
LIMIT 500
"""

print("Results")
x=run_query(queryString)



Results
[('countryName', 'Austria'), ('replacedCurrency', 'http://www.wikidata.org/entity/Q208730'), ('currencyName', 'Austrian schilling')]
[('countryName', 'Belgium'), ('replacedCurrency', 'http://www.wikidata.org/entity/Q232415'), ('currencyName', 'Belgian franc')]
[('countryName', 'Cyprus'), ('replacedCurrency', 'http://www.wikidata.org/entity/Q193716'), ('currencyName', 'Cypriot pound')]
[('countryName', 'Estonia'), ('replacedCurrency', 'http://www.wikidata.org/entity/Q129983'), ('currencyName', 'Estonian kroon')]
[('countryName', 'Finland'), ('replacedCurrency', 'http://www.wikidata.org/entity/Q203354'), ('currencyName', 'Finnish markka')]
[('countryName', 'France'), ('replacedCurrency', 'http://www.wikidata.org/entity/Q184172'), ('currencyName', 'French franc')]
[('countryName', 'Germany'), ('replacedCurrency', 'http://www.wikidata.org/entity/Q16068'), ('currencyName', 'Deutsche Mark')]
[('countryName', 'Greece'), ('replacedCurrency', 'http://www.wikidata.org/entity/Q1546443'), 

In [121]:
#I note that Spain and Ireland are not returned in the previous query (Member of UE = 27, Members that uses Euro = 19 but my last query
# returned 17 replaced currecnies, I have to go deeper in this)



In [122]:
#test query 

queryString = """
#return all the countries (node that are instance of node) in the European Union
SELECT DISTINCT ?c ?cn
WHERE {
   wd:Q458 wdt:P527 ?c .
   ?c wdt:P31 wd:Q6256 ;
      sc:name ?cn . 
}
"""

print("Results")
x=run_query(queryString)



Results
[('c', 'http://www.wikidata.org/entity/Q142'), ('cn', 'France')]
[('c', 'http://www.wikidata.org/entity/Q211'), ('cn', 'Latvia')]
[('c', 'http://www.wikidata.org/entity/Q213'), ('cn', 'Czech Republic')]
[('c', 'http://www.wikidata.org/entity/Q214'), ('cn', 'Slovakia')]
[('c', 'http://www.wikidata.org/entity/Q218'), ('cn', 'Romania')]
[('c', 'http://www.wikidata.org/entity/Q219'), ('cn', 'Bulgaria')]
[('c', 'http://www.wikidata.org/entity/Q224'), ('cn', 'Croatia')]
[('c', 'http://www.wikidata.org/entity/Q229'), ('cn', 'Cyprus')]
[('c', 'http://www.wikidata.org/entity/Q233'), ('cn', 'Malta')]
[('c', 'http://www.wikidata.org/entity/Q28'), ('cn', 'Hungary')]
[('c', 'http://www.wikidata.org/entity/Q33'), ('cn', 'Finland')]
[('c', 'http://www.wikidata.org/entity/Q34'), ('cn', 'Sweden')]
[('c', 'http://www.wikidata.org/entity/Q35'), ('cn', 'Denmark')]
[('c', 'http://www.wikidata.org/entity/Q36'), ('cn', 'Poland')]
[('c', 'http://www.wikidata.org/entity/Q38'), ('cn', 'Italy')]
[('c', '

In [123]:
#from the previous query I noted that not all the nodes in the Europian Union are instance of country, for example Ireland, 
#I explore the Ireland node by performing (1)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q22890 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P206'), ('pName', 'located in or next to body of water'), ('obj', 'http://www.wikidata.org/entity/Q209688'), ('oName', "St George's Channel")]
[('p', 'http://www.wikidata.org/prop/direct/P706'), ('pName', 'located in/on physical feature'), ('obj', 'http://www.wikidata.org/entity/Q38272'), ('oName', 'British Isles')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q38272'), ('oName', 'British Isles')]
[('p', 'http://www.wikidata.org/prop/direct/P1792'), ('pName', 'category of associated people'), ('obj', 'http://www.wikidata.org/entity/Q7031850'), ('oName', 'Category:Irish people')]
[('p', 'http://www.wikidata.org/prop/direct/P8744'), ('pName', 'economy of topic'), ('obj', 'http://www.wikidata.org/entity/Q100860333'), ('oName', 'economy of Ireland')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('obj', 'http://www.wikidata.org/en

In [124]:
#from the previous query I discovered that some nodes returned by the previous query about all the memebers in the UE are not all 
#istances of country but are more general nodes. This is the case of Ireland for example that is an istance of island. 
#But I see that is connected with the country property (P17) to another country (Ireland, that is the actual country 
#node associated to Ireland)

Final query for this task

In [125]:
queryString = """
SELECT DISTINCT ?countryName ?replacedCurrency ?currencyName
WHERE {
    #euro replace ?replaceCurrency
   wd:Q4916 wdt:P1365 ?replacedCurrency  .
   ?replacedCurrency sc:name ?currencyName .
   
   #?replacedCurrency was used in a country
   ?replacedCurrency wdt:P17 ?country . 
   
   #check that the country is in the European Union
   FILTER (?country = ?c){
   
        #return all the countries in the European Union
        SELECT DISTINCT ?c
        WHERE {
           wd:Q458 wdt:P527 ?obj .
           ?obj wdt:P17 ?c .
        }
   }
   
   ?country sc:name ?countryName . 
}
ORDER BY ?countryName
LIMIT 500
"""

print("Results")
x=run_query(queryString)




Results
[('countryName', 'Austria'), ('replacedCurrency', 'http://www.wikidata.org/entity/Q208730'), ('currencyName', 'Austrian schilling')]
[('countryName', 'Belgium'), ('replacedCurrency', 'http://www.wikidata.org/entity/Q232415'), ('currencyName', 'Belgian franc')]
[('countryName', 'Cyprus'), ('replacedCurrency', 'http://www.wikidata.org/entity/Q193716'), ('currencyName', 'Cypriot pound')]
[('countryName', 'Estonia'), ('replacedCurrency', 'http://www.wikidata.org/entity/Q129983'), ('currencyName', 'Estonian kroon')]
[('countryName', 'Finland'), ('replacedCurrency', 'http://www.wikidata.org/entity/Q203354'), ('currencyName', 'Finnish markka')]
[('countryName', 'France'), ('replacedCurrency', 'http://www.wikidata.org/entity/Q184172'), ('currencyName', 'French franc')]
[('countryName', 'Germany'), ('replacedCurrency', 'http://www.wikidata.org/entity/Q16068'), ('currencyName', 'Deutsche Mark')]
[('countryName', 'Greece'), ('replacedCurrency', 'http://www.wikidata.org/entity/Q1546443'), 

## Task 5
Return all types of dollar in the world (US Dollar, Australian Dollar, ecc...)

In [126]:
#explore the currency node (obtained during the exploration of Italy node) by performing (1)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q8142 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q11105360'), ('oName', 'medium of exchange')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q47574'), ('oName', 'unit of measurement')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q831772'), ('oName', 'foreign currencies')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q831772'), ('oName', 'foreign currencies')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P17'), ('oName', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q101314624'), ('oName', 'Lean Logic: A Dictionary 

Final query for this task

In [127]:
#search for all the currencies that have a name that contains the keyword "dollar"

queryString = """
SELECT DISTINCT ?currency ?currencyName
WHERE {
    ?currency wdt:P31 wd:Q8142 . 
    ?currency sc:name ?currencyName . 
    FILTER(REGEX(?currencyName, "[D,d]ollar"))
   
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)


Results
[('currency', 'http://www.wikidata.org/entity/Q3712659'), ('currencyName', 'Texas Dollar')]
[('currency', 'http://www.wikidata.org/entity/Q209792'), ('currencyName', 'Jamaican dollar')]
[('currency', 'http://www.wikidata.org/entity/Q1104069'), ('currencyName', 'Canadian dollar')]
[('currency', 'http://www.wikidata.org/entity/Q259502'), ('currencyName', 'Australian dollar')]
[('currency', 'http://www.wikidata.org/entity/Q4597'), ('currencyName', 'Solomon Islands dollar')]
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyName', 'United States dollar')]
[('currency', 'http://www.wikidata.org/entity/Q26365'), ('currencyName', 'Eastern Caribbean dollar')]
[('currency', 'http://www.wikidata.org/entity/Q2411666'), ('currencyName', 'Newfoundland dollar')]
[('currency', 'http://www.wikidata.org/entity/Q242988'), ('currencyName', 'Liberian dollar')]
[('currency', 'http://www.wikidata.org/entity/Q261447'), ('currencyName', 'Straits dollar')]
[('currency', 'http://www.wikid

## Task 6
Return the currencies used in at least 2 different countries (return also the number of countries using each currency).

In [128]:
#for all the countries in the world, group them by the currency used, and then select the currency with at most 2 countries that use them
#I'm considering all the country nodes, so I will exclude a lot of principalities or micro-states

queryString = """
SELECT DISTINCT ?currency ?currencyName (COUNT(DISTINCT ?country) AS ?nCountries)
WHERE {
    #retrieve all the country nodes
    ?country wdt:P31 wd:Q6256 . 
    
    #retrieve the currency for each country retrieved
    ?country wdt:P38 ?currency . 
     
    #name of the currency
    ?currency sc:name ?currencyName 
   
}
GROUP BY ?currency ?currencyName
HAVING (COUNT (DISTINCT ?country) >= 2)
LIMIT 500
"""

print("Results")
x=run_query(queryString)



Results
[('currency', 'http://www.wikidata.org/entity/Q259502'), ('currencyName', 'Australian dollar'), ('nCountries', '4')]
[('currency', 'http://www.wikidata.org/entity/Q26365'), ('currencyName', 'Eastern Caribbean dollar'), ('nCountries', '6')]
[('currency', 'http://www.wikidata.org/entity/Q861690'), ('currencyName', 'West African CFA franc'), ('nCountries', '8')]
[('currency', 'http://www.wikidata.org/entity/Q181907'), ('currencyName', 'South African rand'), ('nCountries', '3')]
[('currency', 'http://www.wikidata.org/entity/Q1472704'), ('currencyName', 'New Zealand dollar'), ('nCountries', '3')]
[('currency', 'http://www.wikidata.org/entity/Q25344'), ('currencyName', 'Swiss franc'), ('nCountries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyName', 'United States dollar'), ('nCountries', '11')]
[('currency', 'http://www.wikidata.org/entity/Q25224'), ('currencyName', 'pound sterling'), ('nCountries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q8477

In [129]:
#results are quite strange for the same cause of the previous query, some nodes connected to the currency node with the "country" property is not an istance of countru 
# So I try this query, that gives better results but I will consider also some micro-states and pricipalities that aren't countries and I will also consider some currency 
# that are no more used


queryString = """
SELECT DISTINCT ?currency ?currencyName (COUNT(DISTINCT ?c) AS ?nCountries)
WHERE {
    #retrieve all the currencies with their name and associated country
    ?currency wdt:P31 wd:Q8142 ; 
              wdt:P17 ?c ;
              sc:name ?currencyName .
}
GROUP BY ?currency ?currencyName
HAVING (COUNT (DISTINCT ?c) >= 2)
ORDER BY ?currencyName
LIMIT 500
"""

print("Results")
x=run_query(queryString)



Results
[('currency', 'http://www.wikidata.org/entity/Q1983857'), ('currencyName', 'Abkhazian apsar'), ('nCountries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q2329625'), ('currencyName', 'Austrian krone'), ('nCountries', '3')]
[('currency', 'http://www.wikidata.org/entity/Q208730'), ('currencyName', 'Austrian schilling'), ('nCountries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q306306'), ('currencyName', 'Austro-Hungarian gulden'), ('nCountries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q804170'), ('currencyName', 'Baiocco'), ('nCountries', '3')]
[('currency', 'http://www.wikidata.org/entity/Q942092'), ('currencyName', 'British West Indies dollar'), ('nCountries', '3')]
[('currency', 'http://www.wikidata.org/entity/Q2348843'), ('currencyName', 'Cambodian tical'), ('nCountries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q737779'), ('currencyName', 'Caribbean guilder'), ('nCountries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q3658911

In [130]:
#I note that probably a lot of currencies are no more used so have been replaced by a new currency, I explore the Lira currency 
#that was replaced by the Euro by performing (1)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q204992 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q28783456'), ('oName', 'obsolete currency')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q172579'), ('oName', 'Kingdom of Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q3181656'), ('oName', 'The Nuttall Encyclopædia')]
[('p', 'http://www.wikidata.org/prop/direct/P1001'), ('pName', 'applies to jurisdiction'), ('obj', 'http://www.wikidata.org/entity/Q38'), ('oName', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q38'), ('oName', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('pName', 'replaced by'), ('obj', 'http://www.wikidata.org/entity/Q4916'), ('oName', 'euro')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'),

In [131]:
#I note that probably a lot of currencies are no more used so have been replaced by a new currency, I explore the Lira currency 
#that was replaced by the Euro by performing (2)

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q204992 .
   FILTER(!isLiteral(?s))
   
   ?p sc:name ?pName.
   ?s sc:name ?sName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1423581'), ('sName', 'Italian Regency of Carnaro'), ('p', 'http://www.wikidata.org/prop/direct/P38'), ('pName', 'currency')]
[('s', 'http://www.wikidata.org/entity/Q548114'), ('sName', 'Free State of Fiume'), ('p', 'http://www.wikidata.org/prop/direct/P38'), ('pName', 'currency')]
[('s', 'http://www.wikidata.org/entity/Q666893'), ('sName', 'Italian Cyrenaica'), ('p', 'http://www.wikidata.org/prop/direct/P38'), ('pName', 'currency')]
[('s', 'http://www.wikidata.org/entity/Q391980'), ('sName', 'Free Territory of Trieste'), ('p', 'http://www.wikidata.org/prop/direct/P38'), ('pName', 'currency')]
[('s', 'http://www.wikidata.org/entity/Q172579'), ('sName', 'Kingdom of Italy'), ('p', 'http://www.wikidata.org/prop/direct/P38'), ('pName', 'currency')]
[('s', 'http://www.wikidata.org/entity/Q4916'), ('sName', 'euro'), ('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pName', 'replaces')]
[('s', 'http://www.wikidata.org/entity/Q153660'), ('sNam

In [132]:
#retrieve all the currencies that have been replaced by another currency
#an old currency is also an instance of obsolete currency 


queryString = """
SELECT DISTINCT ?currency ?currencyName
WHERE {
    ?currency wdt:P31 wd:Q28783456 ; 
              wdt:P1366 ?cur; 
              sc:name ?currencyName .
}
ORDER BY ?currencyName
LIMIT 500
"""

print("Results")
x=run_query(queryString)




Results
[('currency', 'http://www.wikidata.org/entity/Q910712'), ('currencyName', 'Afghan rupee')]
[('currency', 'http://www.wikidata.org/entity/Q4763680'), ('currencyName', 'Anglo-Saxon pound')]
[('currency', 'http://www.wikidata.org/entity/Q783373'), ('currencyName', 'Australian pound')]
[('currency', 'http://www.wikidata.org/entity/Q2329625'), ('currencyName', 'Austrian krone')]
[('currency', 'http://www.wikidata.org/entity/Q160680'), ('currencyName', 'Belarusian ruble')]
[('currency', 'http://www.wikidata.org/entity/Q108555034'), ('currencyName', 'Belarusian ruble')]
[('currency', 'http://www.wikidata.org/entity/Q743823'), ('currencyName', 'British West African pound')]
[('currency', 'http://www.wikidata.org/entity/Q1328829'), ('currencyName', 'Canadian pound')]
[('currency', 'http://www.wikidata.org/entity/Q3931076'), ('currencyName', 'Central American Republic real')]
[('currency', 'http://www.wikidata.org/entity/Q5253462'), ('currencyName', 'Delaware pound')]
[('currency', 'http

In [133]:
#explore the peseta node by performing (1)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q189097 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q541745'), ('oName', 'Captaincy General of Puerto Rico')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q2041543'), ('oName', 'Ottův slovník naučný')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q228'), ('oName', 'Andorra')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q29'), ('oName', 'Spain')]
[('p', 'http://www.wikidata.org/prop/direct/P1001'), ('pName', 'applies to jurisdiction'), ('obj', 'http://www.wikidata.org/entity/Q29'), ('oName', 'Spain')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('pName', 'replaced by'), ('obj', 'http://www.wikidata.org/entity/Q4916'), ('oName', 'euro')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName'

In [134]:
#refine the previous query 

queryString = """
SELECT DISTINCT ?currency ?currencyName (COUNT(DISTINCT ?c) AS ?nCountries)
WHERE {
    #retrieve all the currencies with their name and associated country
    ?currency wdt:P31 wd:Q8142 ; 
              wdt:P17 ?c ;
              sc:name ?currencyName .
    
    #the currency must not be replaced or an instance of obsolete currency
    FILTER NOT EXISTS {?currency wdt:P31 wd:Q28783456} . 
    FILTER NOT EXISTS {?currency wdt:P1366 ?cur} .
    
}
GROUP BY ?currency ?currencyName
HAVING (COUNT (DISTINCT ?c) >= 2)
ORDER BY ?currencyName
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q1983857'), ('currencyName', 'Abkhazian apsar'), ('nCountries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q804170'), ('currencyName', 'Baiocco'), ('nCountries', '3')]
[('currency', 'http://www.wikidata.org/entity/Q2348843'), ('currencyName', 'Cambodian tical'), ('nCountries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q737779'), ('currencyName', 'Caribbean guilder'), ('nCountries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q3658911'), ('currencyName', 'Carlino'), ('nCountries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q847739'), ('currencyName', 'Central African CFA franc'), ('nCountries', '6')]
[('currency', 'http://www.wikidata.org/entity/Q26365'), ('currencyName', 'Eastern Caribbean dollar'), ('nCountries', '8')]
[('currency', 'http://www.wikidata.org/entity/Q5527593'), ('currencyName', 'Gaucho'), ('nCountries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q5639749'), ('currencyName'

In [135]:
#explore one of the previous currency Baiocco (Q804170) by performing (1)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q804170 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q1072140'), ('oName', 'Roman Republic')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q8142'), ('oName', 'currency')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q175881'), ('oName', 'Roman Republic')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q170174'), ('oName', 'Papal States')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q55325323'), ('oName', 'Category:Baiocco')]
5


Final query for this task


In [136]:
#this is the straightforward query for the task: I'm starting retrieving from the country nodes. 
#I will filter by not considering replaced currencies and also obsolete currencies  but there are some problems of incomplete graph, some information are missing
#so I cannot directly filter the results. 
#for all the countries in the world, group them by the currency used, and then select the currencies with at most 2 countries that use them
#but I'm not so happy about the results

queryString = """
SELECT DISTINCT ?currency ?currencyName (COUNT(DISTINCT ?country) AS ?nCountries)
WHERE {
    #retrieve all the country nodes
    ?country wdt:P31 wd:Q6256 . 
    
    #retrieve the currency for each country retrieved
    ?country wdt:P38 ?currency . 
     
    ?currency wdt:P31 wd:Q8142 .
    
    #the currency must not be replaced or an instance of obsolete currency
    FILTER NOT EXISTS {?currency wdt:P31 wd:Q28783456} . 
    FILTER NOT EXISTS {?currency wdt:P1366 ?cur} .
   
    #name of the currency
    ?currency sc:name ?currencyName 
    
}
GROUP BY ?currency ?currencyName
HAVING (COUNT (DISTINCT ?country) >= 2)
LIMIT 500
"""

print("Results")
x=run_query(queryString)






Results
[('currency', 'http://www.wikidata.org/entity/Q259502'), ('currencyName', 'Australian dollar'), ('nCountries', '4')]
[('currency', 'http://www.wikidata.org/entity/Q26365'), ('currencyName', 'Eastern Caribbean dollar'), ('nCountries', '6')]
[('currency', 'http://www.wikidata.org/entity/Q861690'), ('currencyName', 'West African CFA franc'), ('nCountries', '8')]
[('currency', 'http://www.wikidata.org/entity/Q181907'), ('currencyName', 'South African rand'), ('nCountries', '3')]
[('currency', 'http://www.wikidata.org/entity/Q1472704'), ('currencyName', 'New Zealand dollar'), ('nCountries', '3')]
[('currency', 'http://www.wikidata.org/entity/Q25344'), ('currencyName', 'Swiss franc'), ('nCountries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyName', 'United States dollar'), ('nCountries', '11')]
[('currency', 'http://www.wikidata.org/entity/Q25224'), ('currencyName', 'pound sterling'), ('nCountries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q8477

## Task 7
Consider the currencies obtained in task 6, for each currency return the number of people using that currency (hint: consider the maximum popolation available for each country).

In [137]:
#explore the country class by performing (3) and check if there is an info associated to the number of people in that country

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q6256 ?p ?o .
   ?p sc:name ?pName .
   ?o sc:name ?oName .
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)




Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P2046'), ('oName', 'area')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P3067'), ('oName', 'GS1 country code')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P78'), ('oName', 'top-level Internet domain')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P935'), ('oName', 'Commons gallery')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P30'), ('oName', 'continent')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P2936'), ('oName', 'language

In [138]:
#isolate the population data property

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q6256 ?p ?o .
   ?p sc:name ?pName .
   FILTER(REGEX(?oName, "pop"))
   ?o sc:name ?oName .
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1082'), ('oName', 'population')]
1


In [139]:
#use the popluation property for the Italy node 


queryString = """
SELECT DISTINCT ?population
WHERE {
   wd:Q38 wdt:P1082 ?population .
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)




Results
[('population', '60317000')]
1


Final query for this task

In [140]:
#final query

queryString = """
SELECT DISTINCT ?currency ?currencyName (SUM(?population) as ?people)
WHERE {
    #retrieve all the country nodes, their currency, and population
    ?country wdt:P31 wd:Q6256 ;
             wdt:P38 ?currency ;
             wdt:P1082 ?population .
     
    #name of the currency
    ?currency sc:name ?currencyName .
    
    ?currency wdt:P31 wd:Q8142 .
    
    #the currency must not be replaced or an instance of obsolete currency
    FILTER NOT EXISTS {?currency wdt:P31 wd:Q28783456} . 
    FILTER NOT EXISTS {?currency wdt:P1366 ?cur} .
   
    #name of the currency
    ?currency sc:name ?currencyName 
              
}
GROUP BY ?currency ?currencyName
HAVING (COUNT (DISTINCT ?country) >= 2)
LIMIT 500
"""

print("Results")
x=run_query(queryString)

#respect the previous query we have 11 result rows instead of 12, probably one country hasn't the population property and it is not considered. This is an incomplete graph problem.  


Results
[('currency', 'http://www.wikidata.org/entity/Q259502'), ('currencyName', 'Australian dollar'), ('people', '24656582')]
[('currency', 'http://www.wikidata.org/entity/Q26365'), ('currencyName', 'Eastern Caribbean dollar'), ('people', '627848')]
[('currency', 'http://www.wikidata.org/entity/Q861690'), ('currencyName', 'West African CFA franc'), ('people', '121487314')]
[('currency', 'http://www.wikidata.org/entity/Q181907'), ('currencyName', 'South African rand'), ('people', '64389483')]
[('currency', 'http://www.wikidata.org/entity/Q1472704'), ('currencyName', 'New Zealand dollar'), ('people', '5131346')]
[('currency', 'http://www.wikidata.org/entity/Q25344'), ('currencyName', 'Swiss franc'), ('people', '8503939')]
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyName', 'United States dollar'), ('people', '377344239')]
[('currency', 'http://www.wikidata.org/entity/Q25224'), ('currencyName', 'pound sterling'), ('people', '69135273')]
[('currency', 'http://www.wiki